In [174]:
# Importing necessary libraries

# !pip install pydot
# !pip install pydotplus
# !pip install graphviz
# !python -m pip install -U matplotlib
# !pip install pycountry
# !pip install chart_studio
# !pip install cufflinks
# %matplotlib inline
# !pip install bar_chart_race
# !pip install plotly.subplots
# !pip install colorspacious
# !pip install wordcloud
# !pip install wordcloud
# !pip install pyspellchecker
# !pip install spacy
# !pip install scispacy
# !pip install textblob
# !pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.0/en_core_sci_lg-0.5.0.tar.gz

import pandas as pd
import numpy as np
import time
from datetime import datetime
import sys
from scipy import stats
from bs4 import BeautifulSoup
import requests
import bz2
import collections
import itertools
import os
import random
import h5py
import re
import urllib.parse
import urllib.request
import zipfile
import spacy
import urllib.parse
from scispacy.abbreviation import AbbreviationDetector
nlp = spacy.load("en_core_sci_lg")
nlp.add_pipe("abbreviation_detector")
import scispacy
from tqdm import tqdm
import bar_chart_race as bcr
from dateutil.parser import parse
from IPython.display import HTML
from spellchecker import SpellChecker
from textblob import TextBlob
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from IPython.display import display
from statistics import mode
from pandas import set_option
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import pydot
import pydotplus
import graphviz
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.offline as pyo
import pycountry
import plotly.express as px
import plotly.offline as py
from plotly.offline import download_plotlyjs,init_notebook_mode,plot,iplot
import chart_studio.plotly as pc
import cufflinks
cufflinks.go_offline()
cufflinks.set_config_file(world_readable=True, theme='pearl')
init_notebook_mode(connected = True)
import matplotlib.font_manager
import matplotlib as mpl
from matplotlib import cm
from colorspacious import cspace_converter
from wordcloud import WordCloud, STOPWORDS
import seaborn as sns
sns.set()
plt.rcParams['figure.figsize'] = 8, 5
plt.style.use("fivethirtyeight")

# Disable warnings 
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 100)

# 1. Data Extraction

JAVA and Python programming languages were used in conjuction to retrieve all relevant clinical trial records from ClinicalTrials.gov database. The JAVA script is uploaded separately to the Google Drive and stored in the Folder 'JAVA'.
***
The Data Extraction process here starts with importing the search results returned by the JAVA script after interacting with ClinicalTrials.gov RESTful API. Then, Python script written with URL-encoded query is used to scrape the full records of clinical trials from the ClinicalTrials.gov database using Beautiful Soup Library. 

In [ ]:
# Import the dataset
Database = pd.read_excel('Python/Clinical Trials market research 2022-05-16.xlsx')

# Create a list of URLs
Database['link'] = 'https://www.clinicaltrials.gov/ct2/show/record/'+ Database['TRIAL ID (NCT number)']
links = Database['link'].tolist()

# Scrap the full records of clinical trials using Beautiful Soup
df = pd.DataFrame()
page = requests.get('https://www.clinicaltrials.gov/ct2/show/record/NCT05101148') # set an example
soup = BeautifulSoup(page.content, 'html.parser')
x = soup.find_all('table')
table = pd.read_html(str(x))
output = table[2]

# Convert the retrieved records into dataframe format
Trialtable = pd.DataFrame([])
for i in links:
    try:
        page = requests.get(i)
        soup = BeautifulSoup(page.content, 'html.parser')
        x = soup.find_all('table')
        
        if len(x) == 0:
            pass
        
        else:
            table = pd.read_html(str(x))
            output = table[2] # only import the second table
            output.columns = ('Category','Details')
            output = output.dropna(how = 'all') 
            spec_chars = ["!",'"',"#","%","&","'","*","+","-","<","=",">","?","@","\\","//","^","_",
                  "`","{","|","}","~","–","ICMJE"] # remove all special characters in columns
            for char in spec_chars: 
                output['Details'] = output['Details'].str.replace(char, '')
                output['Category'] = output['Category'].str.replace(char, '')

            df = output.T.reset_index()
            df.columns = df.iloc[0] # set column
            df = df.drop(df.index[0])
            df['link']=i
            df = df.rename(columns = lambda x: x.split('(')[0]) # clean and standardize column names
            df = df.loc[~df.index.duplicated(keep = 'first')] # drop duplicated columns
            df = df.loc[:,~df.columns.duplicated()]
            df = df.replace(r"^ +| +$", r"", regex = True) # remove spaces at the opening and end of strings
            df.columns = df.columns.str.replace(r"^ +| +$", r"", regex=True) # remove spaces in column headings
            Trialtable = Trialtable.append(df) 
            
    except Exception:
        None

Trialtable = Trialtable.reset_index().drop(columns = 'index')
Final_df = pd.merge(Database,Trialtable, on = 'link')

# Save a copy of the dataframe in xlsx format
Final_df.to_excel('/project/Python/clinical trials.xlsx') 

# 2. Data Preparation

This purpose of this section is to prepare the dataset for Exploratory Data Analysis. It includes the steps to clean, normalise the text, and extract information from unstructured columns using a combination of Python functionalities and Natural Language Processing (NLP) techniques. 

In [86]:
# Import and load the dataset
dataset = pd.read_excel('/project/Python/clinical trials.xlsx')
dataset.head(5)

,Unnamed: 0,TRIAL ID (NCT number),SPONSOR,STATUS (active/recruiting),CONDITION/DISEASE,INTERVENTION (drug name),BRIEF SUMMARY,Phase,Allocation,Intervention Model,Masking,Primary Purpose,Gender,No of Participants,Age (min) years,Age (max) years,link,Category,First Submitted Date,First Posted Date,Last Update Posted Date,Actual Study Start Date,Estimated Primary Completion Date,Current Primary Outcome Measures,Original Primary Outcome Measures,Change History,Current Secondary Outcome Measures,Original Secondary Outcome Measures,Current Other Prespecified Outcome Measures,Original Other Prespecified Outcome Measures,Descriptive Information,Brief Title,Official Title,Brief Summary,Detailed Description,Study Type,Study Design,Target FollowUp Duration,Biospecimen,Sampling Method,Study Population,Condition,Intervention,Study Groups/Cohorts,Publications,Includes publications given by the data provider as well as publications identified by ClinicalTrials.gov Identifier,Recruitment Information,Recruitment Status,Actual Enrollment,Original Estimated Enrollment,Estimated Study Completion Date,Eligibility Criteria,Sex/Gender,Ages,Accepts Healthy Volunteers,Contacts,Listed Location Countries,Removed Location Countries,Administrative Information,NCT Number,Other Study ID Numbers,Has Data Monitoring Committee,U.S. FDAregulated Product,IPD Sharing Statement,Responsible Party,Study Sponsor,Collaborators,Investigators,PRS Account,Verification Date,Actual Primary Completion Date,Estimated Enrollment,Study Start Date,Estimated Study Start Date,Original Actual Enrollment,Study Phase,Study Arms,Data element required by the International Committee of Medical Journal Editors and the World Health Organization ICTRP,Results First Submitted Date,Results First Posted Date,Original Enrollment,Actual Study Completion Date,Study Completion Date,Clinical Trial Sites
0,7779,NCT00534235,Paradigm Spine,Completed,Spinal Stenosis,"Procedure:Decompression,Device:Posterolateral ...",Post-Approval Clinical Study Comparing the Lon...,Not Applicable,Randomized,Parallel Assignment,Single,Treatment,NaN,322,NaN,NaN,https://www.clinicaltrials.gov/ct2/show/record...,Details,"September 20, 2007","September 24, 2007","February 6, 2020",NaN,NaN,Number of Subjects With Improvement of at Leas...,Not Provided,Scramble access to archive site dont want c...,Number of Subjects With a Decrease in Zurich C...,Not Provided,Mean Rotation (F to E) in Degrees At Level(s)...,Not Provided,Descriptive Information,PostApproval Clinical Trial Comparing the Long...,PostApproval Study to Investigate The Long Ter...,PostApproval Clinical Study Comparing the Long...,The primary objective of this postapproval stu...,Interventional,Allocation: RandomizedIntervention Model: Para...,NaN,NaN,NaN,NaN,Spinal Stenosis,Procedure: Decompression Either microsurgical ...,NaN,"Davis R, Auerbach JD, Bae H, Errico TJ. Can lo...",Includes publications given by the data provid...,Recruitment Information,Completed,322.0,460,NaN,Not Provided,Not Provided,"Child, Adult, Older Adult",No,Contact information is only displayed when the...,Not Provided,United States,Administrative Information,NCT00534235,PS001,Yes,Not Provided,Plan to Share IPD: Undecided,Paradigm Spine,Paradigm Spine,Musculoskeletal Clinical Regulatory Advisers,Not Provided,Paradigm Spine,January 2020,December 2015 (Final data collection date for ...,NaN,October 2012,NaN,NaN,Not Applicable,Active Comparator: Posterolateral Fusion w/Ped...,Data element required by the International Co...,"November 20, 2018","February 27, 2019",NaN,December 2015,NaN,NaN
1,6444,NCT00760825,International Vaccine Institute,Completed,Vibrio Cholerae,Biological:killed bivalent (O1 and O139)whole ...,The purpose of this study is to determine whet...,Phase 4,NaN,Single Group Assignment,None (Open Label),Prevention,All,200,1 Year,40 Years,https://www.clinicaltrials.gov/ct2/show/record...,Details,"September 25, 2008","September 26, 2008","June 17, 2014",NaN,NaN,Proportion of subjec

In [87]:
# Make a copy of the dataset 
df = dataset.copy()

# Inspect the column names
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7979 entries, 0 to 7978
Data columns (total 84 columns):
 #   Column                                                                                                                     Non-Null Count  Dtype  
---  ------                                                                                                                     --------------  -----  
 0   Unnamed: 0                                                                                                                 7979 non-null   int64  
 1   TRIAL ID (NCT number)                                                                                                      7979 non-null   object 
 2   SPONSOR                                                                                                                    7979 non-null   object 
 3   STATUS (active/recruiting)                                                                                                 7979 

In [88]:
# Define a function to remove specified columns
def removecol(df,col):
    df.drop(columns = col, axis = 1, inplace=True)
    
# Define a function to change the type of columns to integer
def columntype_to_integer(df,columns):
    for col in columns:
        df[col] = df[col].astype('int64')

# Define a function to change the type of columns to string
def columntype_to_str(df,columns):
    for col in columns:
        df[col] = df[col].astype(str) 
        
# Define a function to rename column names
def renamecol(df,col_old,col_new):
    df.rename(columns = {col_old:col_new}, inplace = True)
    
# Define a function to clean the single-item lists in columns
def de_list(input_field):
    if isinstance(input_field, list):
        if len(input_field) == 0:
            return None
        elif len(input_field) == 1:
            return input_field[0]+' '
        else:
            return ', '.join(input_field)
    else:
        return input_field
    

### 2.1 Data Cleaning

In [89]:
# Fill the blank cells with NA
df = dataset.fillna("")

# Rename the dataset columns
renamecol(df,'INTERVENTION (drug name)','Intervention Types')
renamecol(df,'Intervention','Intervention Details')


In [90]:
# Drop duplicated columns
removecol(df,'Descriptive Information')
removecol(df,'Category')
removecol(df,"Study Phase")
removecol(df,'Estimated Enrollment')
removecol(df,'Original Actual Enrollment')
removecol(df,'Actual Enrollment')
removecol(df,'Results First Submitted Date')
removecol(df,'Original Estimated Enrollment')
removecol(df,'Original Enrollment')
removecol(df,'Investigators')
removecol(df,'Study Sponsor')
removecol(df,'Responsible Party')
removecol(df,'NCT Number')
removecol(df,'Brief Title')
removecol(df,'Study Design')
removecol(df,'Condition')
removecol(df,'Recruitment Status')
removecol(df,'Sex/Gender')
removecol(df,'Recruitment Information')

# Remove irrelevant substrings in columns
df['Actual Primary Completion Date'] = df['Actual Primary Completion Date'].str.replace(
                                         "\(Final data collection date for primary outcome measure\)",'')
df['Estimated Primary Completion Date'] = df['Estimated Primary Completion Date'].str.replace(
                                            "\(Final data collection date for primary outcome measure\)",'')


In [91]:
# Normalize Date formats
# Get the actual or estimated start dates, primary completion dates and study completion dates

# Start Date
df["Actual/Estimated Study Start Date"] = (df["Actual Study Start Date"]+df["Study Start Date"]+df["Estimated Study Start Date"]
                                           ).apply(parse)  
# Primary completion date
df["Actual/Estimated Primary Completion Date"] = (df['Estimated Primary Completion Date']+df["Actual Primary Completion Date"]
                                                  ).apply(parse)
# Study completion date
df['Actual/Estimated Study Completion Date'] = (df["Actual Study Completion Date"]+df["Study Completion Date"]+df["Estimated Study Completion Date"]
                                                ).str.replace(' ','-').str.replace(',','')

# Assume Study Completion date equals Primary Completion Date if the Study completion date is not provided
df.loc[df['Actual/Estimated Study Completion Date'] == 'Not-Provided',
          'Actual/Estimated Study Completion Date'] = df["Actual/Estimated Primary Completion Date"]
df.loc[df['Actual/Estimated Study Completion Date'] != 'Not-Provided',
          'Actual/Estimated Study Completion Date'] = pd.to_datetime(df["Actual/Estimated Study Completion Date"])


In [92]:
# Drop duplicated study records
df = df.drop_duplicates(subset = ["TRIAL ID (NCT number)", "Actual/Estimated Study Start Date"],
                        keep=False)


In [93]:
# Normalize Age formats 
# Convert continuous variables into cateogorical variable

# Create list of Minimum Age
minagelist = ['0 Years','6 Months', '6 Weeks','1 Month', '12 Months','28 Weeks','35 Days',
             '2 Months', '46 Days', '365 Days','4 Months','27 Weeks', '9 Months', '42 Days',
             '7 Weeks','43 Days', '3 Months', '64 Days', '7 Months','24 Weeks', '60 Days',
             '10 Months','29 Weeks', '8 Weeks', '85 Days', '8 Months', '55 Days','10 Weeks', 
             '40 Weeks', '25 Weeks','61 Days', '26 Weeks', '5 Months', '45 Weeks', '5 Weeks',
             '11 Months', '22 Weeks', '54 Days', '35 Weeks', '40 Days','56 Days','13 Weeks',
             '39 Days', '23 Weeks', '34 Weeks', '70 Days', '31 Days', '16 Weeks', '48 Days', 
             '33 Weeks','76 Days','30 Weeks', '90 Days']

# Create lists of Maximum Age
maxagelist = ['12 Weeks','10 Weeks','36 Weeks','150 Days','90 Days','74 Days',
              '180 Days','12 Months','17 Days','14 Days','3 Days','21 Days',
              '28 Days','6 Months','34 Weeks', '364 Days','17 Weeks','77 Days', 
              '4 Days','40 Days','2 Weeks','30 Days','5 Days','65 Days','7 Weeks', 
              '76 Days','210 Days','196 Days','72 Hours','16 Days','7 Months','9 Months', 
              '64 Days','11 Weeks','29 Weeks','8 Weeks','83 Days','70 Days','24 Hours',
              '120 Days','48 Hours','2 Days','1 Day','119 Days','35 Days','18 Weeks', 
              '89 Days','4 Months','98 Days','182 Days','20 Weeks','8 Days','49 Days',
              '6 Weeks','12 Days','2 Months','14 Weeks','31 Weeks','5 Months','91 Days',
              '27 Days','3 Months','1 Month','13 Days', '7 Days', '32 Weeks', 
              '26 Days','26 Weeks', '42 Days', '40 Weeks', '18 Days', '120 Hours', 
              '365 Days','84 Days','75 Days', '3 Weeks', '33 Weeks','42 Weeks','16 Weeks',
              '60 Days', '36 Days', '300 Days','19 Days','10 Months','13 Weeks', 
              '29 Days','15 Weeks', '56 Days', '44 Days','8 Months', '96 Hours', 
              '72 Days', '59 Days', '12 Hours','4 Weeks', '51 Days', '35 Weeks',
              '36 Hours', '60 Minutes','11 Months','10 Days','41 Weeks','111 Days', 
              '9 Weeks','88 Days','44 Weeks','19 Weeks','28 Weeks''63 Days', '112 Days', 
              '20 Minutes','25 Days', '104 Days','6 Days', '37 Weeks','30 Weeks', '20 Days','2 Hours']
maxagelist_2 = ['< 1 Year','1 Year','15 Years','14 Years','3 Years','21 Years',
                '2 Years','11 Years', '5 Years', '12 Years', '13 Years', '4 Years',
                '6 Years', '10 Years', '9 Years','8 Years', '7 Years']
maxagelist_3 = ['16 Years', '17 Years','70 Years', '50 Years', '90 Years','30 Years',
                '40 Years', '100 Years','60 Years', '80 Years','130 Years', '20 Years', '120 Years']

# Normalize age formats
df.loc[df['Age (min) years'] == '60 Months','Age (min) years'] = '5 Years'
df.loc[df['Age (min) years'] == '30 Months','Age (min) years'] = '3 Years'
df.loc[(df['Age (min) years'].str.contains('|'.join(minagelist), case = False)),'Age (min) years'] = '< 1 Year'

df.loc[df['Age (max) years'] == '60 Months','Age (max) years'] = '5 Years'
df.loc[df['Age (max) years'] == '40 Months','Age (max) years'] = '3 Years'
df.loc[df['Age (max) years'] == '180 Months','Age (max) years'] = '15 Years'
df.loc[df['Age (max) years'] == '30 Months','Age (max) years'] = '3 Years'

df.loc[(df['Age (max) years'].str.contains('|'.join(maxagelist), case = False)),'Age (max) years'] = '< 1 Year'
df.loc[(df['Age (max) years'].str.contains('|'.join(maxagelist_2), case = False)),'Age (max) years'] = '<= 15 Years'
df.loc[(df['Age (max) years'].str.contains('|'.join(maxagelist_3), case = False)),'Age (max) years'] = '> 15 Years'


In [94]:
# Identify the Intervention Types of clinical trials
df['Intervention Types'] = df['Intervention Types']+', '

# Biological 
df['INTERVENTION Biological'] = df['Intervention Types'].astype(str).str.lower().str.findall(
                                   r'Biological:(.*?),',flags = re.IGNORECASE)

# Drug
df['INTERVENTION Drug'] = df['Intervention Types'].astype(str).str.lower().str.findall(
                             r'Drug:(.*?),',flags = re.IGNORECASE)

# Diagnostic Test
df['INTERVENTION Diagnostic Test'] = df['Intervention Types'].astype(str).str.lower().str.findall(
                                        r'Diagnostic Test:(.*?),',flags = re.IGNORECASE)

# Device
df['INTERVENTION Device'] = df['Intervention Types'].astype(str).str.lower().str.findall(
                               r'Device:(.*?),',flags = re.IGNORECASE)\

# Dietary Supplement
df['INTERVENTION Dietary Supplement'] = df['Intervention Types'].astype(str).str.lower().str.findall(
                                           r'Dietary Supplement:(.*?),',flags = re.IGNORECASE)

# Radiation
df['INTERVENTION Radiation'] = df['Intervention Types'].astype(str).str.lower().str.findall(
                                  r'Radiation:(.*?),',flags = re.IGNORECASE)

# Behavioral
df['INTERVENTION Behavioral'] = df['Intervention Types'].astype(str).str.lower().str.findall(
                                   r'Behavioral:(.*?),',flags = re.IGNORECASE)

# De-list the strings in column
for c in df.columns:
    df[c] = df[c].apply(de_list)
    

In [95]:
# Replace blank cells with appropriate values
def fillna_2(columns,value):
    for col in columns:
        df[col] = df[col].fillna('').replace('',value,regex = True)

# Fill null with 'Not Provided'
fillna_2(['Age (min) years','Age (max) years','Gender'],'Not Provided')

In [96]:
# Remove trials that are not for paediatrics 
# Remove trials with missing gender age details

df = df[(df['Age (min) years'] != '18 Years') &
     (df['Age (min) years'] != '17 Years') &
     (df['Age (min) years'] != '16 Years') &
     (df['Gender'] != 'Not Provided') & 
     ((df['Age (min) years'] != 'Not Provided') | (df['Age (max) years'] != '> 15 Years')) & 
     ((df['Age (min) years'] != 'Not Provided') | (df['Age (max) years'] != 'Not Provided'))]


In [97]:
# Remove 'Year' from Age details
df['Age (min) years'] = df['Age (min) years'].str.replace('Years','').str.replace('Year','')
df['Age (max) years'] = df['Age (max) years'].str.replace('Years','')


In [98]:
print('Total columns in the dataset:',len(df))

Total columns in the dataset: 5701


### 2.2 Data Enrichment

In [99]:
# Convert continuous variables into categories
# To aid exploratory data analysis only

# Number of participants
bins = [-1, 50, 100, 150, 200, 250, 300, 600, 25428392]
group_names = ['< 50', '51-100', '101-150', '151-200', '201-250', '251-300', '301-600', '>600']
categories = pd.cut(df['No of Participants'], bins, labels = group_names)
df['Participant size_bins'] = categories

# Study Duration
# Calculate the study duration (in days)
df.loc[df['Actual/Estimated Study Completion Date'] != 'Not-Provided','Study Duration (in days)'] = df['Actual/Estimated Primary Completion Date']-df["Actual/Estimated Study Start Date"]

# Identify the year that the clinical trial commences
df['Year Started'] = pd.DatetimeIndex(df['Actual/Estimated Study Start Date']).year

# Calculate the study duration
df['Study Duration (in days)'] = df['Study Duration (in days)'].astype(str).str.replace(' days','')
df['Study Duration (in days)'] = df['Study Duration (in days)'].astype(int)

# Remove clinical trials with a study duration less than or equal to 0 days
df = df[(df['Study Duration (in days)'] >0)]

# Remove clinical trials with a study completion date before primary completion date
df['date difference'] = df['Actual/Estimated Study Completion Date'] - df['Actual/Estimated Primary Completion Date']
df['date difference'] = df['date difference'].astype(str).str.replace(' days','')
df['date difference'] = df['date difference'].astype(int)
df = df[(df['date difference'] >=0)]

# Study Duration in years
bins = [-1, 365, 730, 1095, 1460, 1825, 20000]
group_names = ['<1 year', '1-2 years', '2-3 years', '3-4 years', '4-5 years', '> 5 years']
categories = pd.cut(df['Study Duration (in days)'], bins, labels = group_names)
df['Study Duration_bins'] = categories


In [100]:
# Identify the number of Study Arms 

# Create a list of the Study Arms
Arms = ['experimental', 'active comparator', 'placebo Comparator', 'sham comparator']

pattern = '|'.join(Arms).lower()

# Label the Arm Type
df['Arm Type'] = df['Study Arms'].astype(str).str.lower().str.findall(
                 pattern,flags = re.IGNORECASE).str.join(', ').apply(lambda x: ', '.join(list(set(str(x).split(', ')))))

# Calculate the number of arms
df['Number of Arms'] = df['Arm Type'].apply(lambda x: len(x.split(", "))).astype(int)

# Replace null values with 0
df['Number of Arms'] = df['Number of Arms'].fillna('').replace('','0',regex=True) 


In [101]:
# Identify the number of clincial trial sites
df['Number of Clinical Trial Sites'] = df['Clinical Trial Sites'].apply(lambda x: len(x.split(",")))


In [102]:
# Identify the number of countries 
df['Listed Location Countries'] = df['Listed Location Countries'].str.replace('  ',' ').replace(' ,',',').replace(', ',',').replace(' , ',',')
df['Number of countries'] = df['Listed Location Countries'].apply(lambda x: len(x.split(", ")))


In [103]:
# Convert continuous variables into categories
# To aid Exploratory Data Analysis only

# Number of clinical trial sites
bins = [-1, 10, 20, 30, 40, 50, 60, 1000]
group_names = ['< 10', '11-20', '21-30', '31-40', '41-50', '51-60', '>60']
categories = pd.cut(df['Number of Clinical Trial Sites'], bins, labels=group_names)
df['Number of Clinical Trial Sites_bins'] = categories

# Number of countries that the trial takes place
bins = [-1, 1, 2, 3, 4, 5, 6, 7,8,9,10, 1000]
group_names = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '>10']
categories = pd.cut(df['Number of countries'], bins, labels=group_names)
df['Number of countries_bins'] = categories


In [104]:
# Create Binary columns for each listed country location
df = pd.concat([df, df['Listed Location Countries'].str.get_dummies(sep = ', ')], axis = 1)
df = df.drop(columns = ['The Democratic Republic of the','The Former Yugoslav Republic of','Republic of','Not Provided'])


### 2.3 Further Data Processing

In [105]:
# Create a new column that aggregates the 11 free-text columns
df['Free Text_'] = (df['Intervention Details'].astype(str)+' '+
                    df['Study Arms'].astype(str)+' '+
                    df['Detailed Description'].astype(str)+' '+
                    df['BRIEF SUMMARY'].astype(str)+' '+
                    df['Official Title'].astype(str)+' '+
                    df['Current Primary Outcome Measures'].astype(str)+' '+
                    df['Original Primary Outcome Measures'].astype(str)+' '+
                    df['Current Secondary Outcome Measures'].astype(str)+' '+
                    df['Original Secondary Outcome Measures'].astype(str)+' '+
                    df['Current Other Prespecified Outcome Measures'].astype(str)+' '+
                    df['Original Other Prespecified Outcome Measures'].astype(str)
                    ).str.lower().replace(':','',regex = True) # turn all text to lowercase

In [106]:
# Define a function to clean the text using Python functionalities and NLP techniques
def clean(text):
    # removing paragraph numbers
    text = re.sub('[0-9]+.\t','',str(text))
    # removing new line characters
    text = re.sub('\n ','',str(text))
    text = re.sub('\n',' ',str(text))
    # removing apostrophes
    text = re.sub("'s",'',str(text))
    # removing special characters    
    text = re.sub('\"','',str(text))
    text = re.sub("[\(\[].*?[\)\]]", "", str(text))
    text = re.sub(r'([a-z])([A-Z])',r'\1 \2',str(text)) # Split combined words
    text = re.sub(r"\s+[a-zA-Z]\s+", ' ', str(text)) # Remove special characters
    text = text.lower()

    # replacing abbreviations with full names
    text = re.sub("aids ", "immunodeficiency syndrome", str(text))
    text = re.sub("hiv ", "human immunodeficiency virus", str(text))
    text = re.sub("adhd ", "Attention deficit hyperactivity disorder", str(text))
    text = re.sub("gvhd ", "Graft versus host disease", str(text))
    text = re.sub("cmv ", "Cytomegalovirus", str(text))
    text = re.sub("cns ", "Central Nervous System", str(text))
    text = re.sub("hpv ", "Human Papilloma Virus", str(text))
    text = re.sub("mps ", "Mucopolysaccharidosis", str(text))
    
    # decode other abbreviations using NLP spaCy library
    doc = nlp(text) 
    altered_tok = [tok.text for tok in doc] 
    for abrv in doc._.abbreviations:
        altered_tok[abrv.start]=str(abrv._.long_form)
    cleaned_sentence =" ".join(altered_tok)
    return cleaned_sentence

# Clean the disease names
df['CONDITION/DISEASE_cleaned'] = df['CONDITION/DISEASE'].apply(clean).str.lower()
# Clean the free text
df['Free Text'] = df['Free Text_'].apply(clean).str.lower()


#### 2.3.1 Categorization

In [107]:
# Categorize Sponsors and Collaborators

# Create a list that includes the Top 50 Pharmaceutical Companies ranked by Pharm Exec
big_pharmacies = [ 'Sinopharm','Roche','Novartis','Merck','AbbVie','Janssen','neurogene',
                   'GlaxoSmithKline','Bristol Myers Squibb','Pfizer','Sanofi','JCR Pharma',
                   'Takeda Pharmaceutical','AstraZeneca','Gilead','Eli Lilly','Amgen',
                   'Bayer','Novo Nordisk','Boehringer Ingelheim','Teva Pharmaceutical',
                   'Merck KGaA','Biogen','Viatris','Astellas Pharma','Daiichi Sankyo',
                   'Otsuka Holdings','CSL','Regeneron Pharmaceuticals','Bausch Health',
                   'Chugai Pharmaceutical','Eisai','Vertex Pharmaceuticals','UCB','Grifols',
                   'Alexion Pharmaceuticals','Servier','Sumitomo Dainippon Pharma',
                   'Sun Pharmaceutical Industries','Abbott','Menarin','Jiangsu Hengrui Medicine',
                   'Sino Biopharmaceutical','Shionogi','Aurobindo Pharma','Catalent','Ipsen',
                   'Endo|H. Lundbeck','Incyte','Jazz Pharmaceuticals','Dr. Reddy’s Laboratories']

pattern = '|'.join(big_pharmacies).lower()

# Replace abbreviations with Full Name
df['SPONSOR'] = df['SPONSOR'].str.replace('GSK','GlaxoSmithKline')

# Label the the name of sponsor
df['Big Names Sponsors'] = df['SPONSOR'].astype(str).str.lower().str.findall(
                           pattern,flags = re.IGNORECASE).str.join(', ').apply(
                           lambda x: ', '.join(list(set(str(x).split(', ')))))

# Label the the name of collaborator
df['Big Names Collaborators'] = df['Collaborators'].astype(str).str.lower().str.findall(
                                pattern,flags = re.IGNORECASE).str.join(', ').apply(
                                lambda x: ', '.join(list(set(str(x).split(', ')))))

# Replace null values with 'No'
df['Big Names Sponsors'] = df['Big Names Sponsors'].fillna('').replace('','No',regex = True)

df['Big Names Collaborators'] = df['Big Names Collaborators'].fillna('').replace('','No',regex = True)

# Inspect the results and make modifications 
df['Big Names Sponsors'] = df['Big Names Sponsors'].str.replace(
                              'viatris, pfizer','viatris') # Sponsor changed from pfizer to viatris for this trial

# Create columns to denote clinical trials with or without the involvement of big pharmaceutical companies
df.loc[df['Big Names Sponsors'] != 'No','Big Names Sponsors (Y/N)'] = 'Yes'
df.loc[df['Big Names Sponsors (Y/N)'] != 'Yes','Big Names Sponsors (Y/N)'] = 'No'
df.loc[df['Big Names Collaborators'] != 'No','Big Names Collaborators (Y/N)'] = 'Yes'
df.loc[df['Big Names Collaborators (Y/N)'] != 'Yes','Big Names Collaborators (Y/N)'] = 'No'


In [108]:
# Categorize Diseases
# The names of diseases were mapped with minor improvisions to the corresponding chapter headings 
# of International Classification of Diseases (ICD-11)

# Ear diseases
Ear = list(map(str.lower,['Otitis media','Ear','hearing','deaf','mastoid','otitis externa',
                         'myringitis','tymppanosclerosis','eustachain tube','mastoiditis',
                         'presbycusis','hereditary','eyelid','lacrimal apparatus','conjunctiva',
                         'cornea','anterior chamber','anterior uvea','pupil','sclera', 'retina',
                         'contact lens','chorioretinal','conjunctival blebitis','bullous aphakic keratopathy',
                         'colour vision','light sensitivity','binocular']))

# Eye diseases
Eye = list(map(str.lower,['Retinal','Stargard','Myopia','lens','optic','eye','vision','blind',
                         'strabismus','ocular','eyeball','glaucoma','vestibular','labyrinthine fistula',
                         'otosclerosis']))

# covid-19
Covid_19 = list(map(str.lower,['Coronavirus','Covid']))

# developmental disorders
Developmental_disorders = list(map(str.lower,['Neurofibromatosis', 'Genetic Disease','Down syndrome','Achondroplasia','cephalocele,'
                                            'congenital','developmental anomalies','spina bifida','anencephaly','iniencephaly',
                                            'amyelencephaly','craniorachischisis','dandy-walker','hypoplasia','focal cerebellar',
                                            'microcephaly','megalencephaly','holoprosencephaly','arrhinencephaly','corpus callosum',
                                            'encephaloclastic disorder','arnold-chiari','development anomalies','gene mutation',
                                            'chromosome','X-linked']))

# Infections and parasitic diseases
Infectious_parasitic =list(map(str.lower,['Streptococcal','Tetanus','Tuberculosis','poliomyelitis','Tetanus','Varicella','Dravet Syndrome',
                                          'Meningococcal','Haemophilus','Foot and Mouth Disease', 'Enterovirus','Cytomegalovirus','Bacterial','Varicella',
                                          'chickenpox','virus','Infection','pathogens','malaria','Sepsis','immunodeficiency syndrome','staphylococcal',
                                          'streptococcal','dengue','zoonotic viral','influenza','viral hepatitis','mycoses','parasitic',
                                          'Human Immunodeficiency Virus','Human Papilloma Virus','Genital Wart']))

# Mental Health and Behavior disorders
MentalHealth_behaviordisorders = list(map(str.lower,['sleep','Depressive Disorder','emotion','depression','anxiety','schizophrenia',
                                                     'mental health','stress','Dementia','Psychological','Depression','psychotic',
                                                     'schizophrenia','adolescent behavior','psychosis','personality disorder','parasomnia'
                                                     'addiction','suicide','anxiety','eating disorder','drug abuse',' mood ','insomnia',
                                                     'learning disabilities','bipolar disorder','autistic spectrum disorder','hypersomnolence',
                                                     'cognitive function','Fragile X Syndrome','cognitive behaviour','control disorder',
                                                     'factitious disorder']))

# Metabolic and Endocrine diseases
Metabolic_endocrine_nutrition = list(map(str.lower,['Mucopolysaccharidosis','Tyrosinemia','Diabetes Mellitus','diabete','metabolism','pineal',
                                                   'parathyroid','Metabolic','pituitary','adrenal gland','endocrine','glucose','puberty','thyroid',
                                                   'eating','nutrition','diet','health','overweight','undernutrition','nutrient','metabolite','Gaucher Disease',
                                                   'body image','obesity','growth','weight','food absorption','polyglandular','Glomerulonephritis',
                                                   'hypothyroidism','hypoinsulinaemia','hypoparathyroidism','hypopituitarism','lymphoid thyroiditis',
                                                   'testicular','adrenocortical','allergy','allergic','allergies','phenylketonuria',
                                                   'Niemann-Pick Disease']))

# Musculoskeletal and multi-system inflammatory disorders
musculoskeletal_inflammatorydisorders=list(map(str.lower,['ankle','osteoporosis', 'osteoarthritis','muscular','muscle','skeletal','arthopathies',
                                                          'tooth','musculoskeletal','cartilage','fibromyalgia','bone','Lupus','soft tissue disorder',
                                                          'Arthritis','immunization','immunogenicity','inflammation','Immunodeficiency','osteopathies',
                                                          'Immunodeficiencies','immune deficiency','rheumatoid arthritis','connective tissue',
                                                          'autoimmune disease','chondropathies','spondylopathies','join disorders','spondyloarthritis',
                                                          'vertebra','juvenile idiopathic','psoriatic arthritis','rheumatoid arthritis',' gout ',
                                                          'paget disease','epiphyseal arrest','hypertrophy','osteomalacia']))

# gastrointestinal diseases
Gastrointestinal = list(map(str.lower, ['Necrotizing Enterocolitis','gastro','gastric','stomach','Urinary','appendicitis',
                                        'Exocrine Pancreatic','Colitis','Hepatitis','Ulcerative Colitis','bowel','hepatic failure',
                                        'Diarrhea','Abdominal','Inflammatory bowel','pancreas','gut','bowel disease','Crohn',
                                        'oesophagus','digestive system','liver','colon','gastrointestinal','Gastroenteritis','coeliac',
                                        'constipation','Hepatitis','hyperoxaluria','Eosinophilic Esophagitis','duodenum','hernias',
                                        'peritonitis','oesophagus','intestine','anal canal','gallbladder', 'biliary tract',
                                        'orofacial complex']))


# Reprodutive health and child birth diseases
Reproductivehealth_childbirth = list(map(str.lower,['pregnancy','pregnant','fertility','contraception','abortion','fertilisation',
                                                    'breast feeding','birth','antenatal care','labour','postnatal care','infant',
                                                    'newborn','prematurity','childbirth','maternal care','fetal growth','breast milk',
                                                    'placental disorder']))

# Respiratory diseases
Respiratory = list(map(str.lower,['Cystic Fibrosis','SARS','Bronchopulmonary Dysplasia','bronchitis','mediastinal','diaphragm',
                                  'Asthma','chronic obstructive pulmonary disease','bronchiolitis','epiglottis''pleural','cystic fibrosis',
                                  'COPD','respiratory','respiration','larynx','vocal cord','trachea','lung','pharynx','Tuberous Sclerosis Complex',
                                  'nasopharyngitis','sinusitis','tonsillitis','laryngopharyngitis','tracheitis','epiglottitis',
                                  'nasal','sinus syndrome','tonsil','adenoid','bronchitis','emphysema','bronchiectasis','pneumonia',
                                  'pyothorax','mediastinum','fibrothorax','chylous effusion','diliary dyskinesia',
                                  'tracheostomy']))

# Skin diseases nd Allergies
Skin = list(map(str.lower,['tinea','warts','Epidermolysis Bullosa','Dermatitis','Skin disorder','stevens-johnson'
                           'Dermatological','psoriasis','eczema','acne','Atopic Dermatitis','stevens johnson',
                           'Molluscum Contagiosum','skin infection','scarring','skin','urticaria','erythema',
                           'angioedema','erythematosus','papulosquamous dermatoses','pemphigus','pemphigold',
                           'collagen','cutaneous cyst','granulocyte']))

# genitourinary diseases
Genitourinary = list(map(str.lower,['Kidney disease','uterus','bartholin gland','salpingitis','oophoritis','vulvitis','genital',
                                    'perinatal period','neonatal period','pelvic organ','gestation','ovum','ovarian','pelvic floor',
                                    'menstrual cycle','genital organ','pelvic pain','permenstrual','ovaries','vagina','vaginal',
                                    'vulva','dysmenorrohoea','endometriosis','adenomyosis','uterine','menopausal','puerperium',
                                    'proteinuria','obstetric haemorrhage','menstruation','menopause','anovulatory bleeding','dyspareunia',
                                    'nonmenstrual bleeding']))

# Neurological and Spinal diseases
Neurological_Spinal = list(map(str.lower,['Epilepsy','Stroke','brain','Autism','Glioblastoma','neurocognitive','movement',
                                         'Glioma','Meningitis','Ependymoma','spinal','Arthritis','Tourette Syndrome','cerebrovascular',
                                         'Hematoma','Vertebral fractures','Migraine','Leigh','Cerebral Palsy','consciousness',
                                         'Anesthesia','hyperkinetic','spongiform encephalopathies','Parkinson','cerebral palsy',
                                         'neurodegenerative','nerve','nervous system','anaesthesia','sclerosis','human prion',
                                         'hyperactivity','Attention deficit','Neurological','Neuro','Dementias',
                                         'Alzheimer','Lennox Gastaut Syndrome','epilepsy','sclerosis','dravet','porencephalic',
                                         'choreiform','dystonic','ataxic','myoclonic','myelitis','myelopathy','porencephalic',
                                         'reye syndrome','pachymeningitis','encephalopathy','encephalitis','paraneoplastic',
                                         'cerebrospinal fluid otorrhoea','intracranial','syringomyelia','Myalgic Encephalopathy',
                                         'Chronic Fatigue Syndrome','Adrenoleukodystrophy']))
                                 
# Blood and cardiovascular diseases
Blood_and_Cardiovascular = list(map(str.lower,['Hypertension','cardiac','cardio','valve','vein ','spleen','fibrinolytic defects','haematopoietic',
                                               'lymphatic vessels','lymph nodes','Cerebrovascular','arteries', 'arterioles','coagulation',
                                               'capillaries','heart','Cardiovascular','Atherosclerosis','pulmonary hypertension','purpura',
                                               'vasculitis','Sickle Cell','Thrombocytopenic','Thrombosis','Thrombotic','Venous','haemorrhagic',
                                               'Neutropenia','iron','arteriosus','cholesterol','Homozygous Familial Hypercholesterolemia',
                                               'Hematologic','Haemophilia','Hemostasis','Hemophilia','Hemoglobinopathies','artery','hypotension',
                                               'hemolytic uremic syndrome','bleeding disorder','Venous Thromboembolism','anaemia','thalassemia',
                                               'Graft versus host disease','graft versus','Blood','Haematological diseases','Anemia','erythrocyte',
                                               'thromboses','venous embolism','platelets','erythrocytes','Aortic Stenosis','blood clot',
                                               'Thrombocytosis','Von Willebrand',]))

# Neoplasms and cancer
Neoplasm_Cancer = list(map(str.lower,['neoplasm','Synovial Sarcoma','osteosarcoma','Neuroendocrine Tumor','cancer',
                                      'Myeloma','Melanoma','Medulloblastoma','Leukemia','Nasopharyngeal','Lymphoblastic',
                                      'Hodgkin','Ewing Sarcoma','Carcinoma','Lymphoma','leukaemia','lymphocyte',
                                      'Tumour','Tumor','Neuroblastoma','Myelodysplastic','Soft Tissue Sarcoma']))

# Define the function to map the disease names                             
def func(disease,new_col):
    df.loc[(df['CONDITION/DISEASE_cleaned'].astype(str).str.contains('|'.join(disease), case = False)),'Disease Type'] = new_col

def diseasetype():
    func(Ear,'Ear Disease')
    func(Eye,'Eye Disease')
    func(Skin,'Skin Disease')
    func(Covid_19,'Covid-19')
    func(MentalHealth_behaviordisorders,'Mental Health Disease and Behavioral Disorder')
    func(Reproductivehealth_childbirth,'Reproductive health and childbirth Disease')
    func(Developmental_disorders,'Developmental Disorder')
    func(Metabolic_endocrine_nutrition,'Endocrine, Nutritional and Metabolic Disease')
    func(Genitourinary,'Genitourinary Disease')
    func(Neurological_Spinal,'Neurological and Spinal Disease')
    func(musculoskeletal_inflammatorydisorders,'Musculoskeletal and Multi-system Inflammatory Disorder')
    func(Gastrointestinal,'Gastrointestinal Disease')
    func(Infectious_parasitic,'General Infectious or Parasitic Disease')
    func(Respiratory,'Respiratory Disease')
    func(Blood_and_Cardiovascular,'Blood and Cardiovascular Disease')
    func(Neoplasm_Cancer,'Neoplasm and Cancer')

diseasetype()


#### 2.3.2 Keyword Extraction 

In [109]:
# Create dictionaries to replace abbreviations by full name and words that share similar meanings by one word

# Outcome Measures
outcome_measures = {'ediary':' patient diary ','diary ':' patient diary ', 'diary cards ' :'patient diary ',
                    'diary card ':' patient diary ','diaries':' patient diary ','journal':' patient diary ',
                    'symptom diary':' patient diary ','survey':' questionnaire', ' interview ':'questionnaire',
                    'scales':'scale',' rating scale': ' questionnaire','subscale':' questionnaire',
                    'checklist':' questionnaire','checklist':' questionnaire','\b(questionnaire)\b':' questionnaire',
                    'visual analogue scale': ' questionnaire'}

# Intervention Procedures
interventional_procedures = { 'samples': 'sample',' mri ':'magnetic resonance imaging','apheresis': 'aphaeresis',
                              'imaging':'magnetic resonance imaging',' ekg ': 'electrocardiogram', 
                              'electrocardiography':'electrocardiogram','electrocardio':'electrocardiogram', 
                              'electrocardiogramgram':'electrocardiogram','haematology':'blood collection', 
                              ' cxr ':'xray', 'x-ray':'xray','angiography': 'xray','Videofluoroscopy' :'xray',
                              'valve replacement': 'surgery','surgical biopsy': 'surgery','bone grafting': 'surgery',
                              'bone graft': 'surgery','transplantation': 'surgery','autografting': 'surgery',
                              'autograft': 'surgery','surgeries':'surgery','ACL reconstruction':'surgery',
                              'surgical reconstruction':'surgery','tendon graft':'surgery',
                              'tendon grafting':'surgery','microfracture':'surgery','craniectomy':'surgery',
                              'craniotomy':'surgery','epiphysiodesis':'surgery','nissen fundoplication':'surgery',
                              'phacoemulsification':'surgery','skin graft':'surgery','skin grafting':'surgery',
                              ' phaco ':'surgery','echocardiogramgram':'echocardiogram','echocardio' : 'echocardiogram',
                              ' ecg ':'echocardiogram','echocardio':'echocardiogram','clinical assessment':'blood collection',
                              'laboratory assessment':'blood collection','clinical assessment':'blood collection',
                              'urine collection':'urinalysis','urine sample collection':'urinalysis',
                              'urine test':'urinalysis',' meg ':'magnetoencephalogram',' eeg ':'electroencephalogram',
                              ' emg ':'electromyography','Ophthalmologic examination':'physical examination',
                              'spinal tap':'lumbar puncture',' lp ':'lumbar puncture','spirometry':'pulmonary function test',
                              ' pet ':'positron emission tomography scan','psychological test': 'cognitive test',
                              ' savr ':'surgical aortic valve replacement','blood draw':' blood collection ',
                              'collect blood':' blood collection ','collecting blood':' blood collection ',
                              'draw blood':' blood collection ','drawing blood':' blood collection ',
                              'blood sampling':' blood collection ','blood collection' :'blood collection ',
                              'blood sample':' blood collection ','serum chemistry':' blood collection ',
                              'pharmacokinetics':'blood collection','blood test':' blood collection ',
                              'withdraw of blood':' blood collection ','coagulation':'blood collection',
                              'withdraw blood':' blood collection ','lung function measurement':' blood collection ',
                              'leucapheresis ':' blood collection ','leukapheresis' :' blood collection ', 
                              'venepuncture': ' blood collection ',' blood sample collection ': ' blood collection ',
                              'phlebotomy': ' blood collection ','x ray':'xray', 'radiograph':'xray',
                              '6minutewalk':'6 minute walk','6mwt':'6 minute walk', 'sixminute':'6 minute walk',
                              'sixmwt':'6 minute walk','stand test':'sit to stand test','implant placement':'implantation',
                              'body fluid' : 'Body fluid collection','saliva collection' : 'Body fluid collection',
                              'collect saliva' : 'Body fluid collection','nasal swab sampling': 'Body fluid collection',
                              ' svf ':'Stromal Vascular Fraction','prosthetic treatment':'Prosthodontics',
                              'dental prosthetics' :'Prosthodontics' ,'hematology':'blood collection', 
                              ' tace ':'Transarterial chemoembolization','light therapy':'phototherapy',
                              ' eam ':'electroanatomic mapping', 'electro-anatomic mapping':'electroanatomic mapping',
                              }

# Drug Delivery Tools
drug_delivery = {'inhale': 'inhalation','eye drop':'ocular','topical':'cutaneous',
                  'swallow': 'oral',' anal ': 'vaginal','pulmonary':'inhalation',
                  'intramuscular':'injection','rectum' : 'vaginal','subcutaneous':'injection',
                  'infusion':'injection','intrathecal':'injection','intravenous':'injection',
                  'nebulisation':'nebulization','nebulizer':'nebulization'
                  }

# Make word replacements
for i, j in interventional_procedures.items():
    df['Free Text'] = df['Free Text'].str.replace(i, j)
for x, y in drug_delivery.items():
    df['Free Text'] = df['Free Text'].str.replace(x, y)
for a, b in outcome_measures.items():
    df['Free Text'] = df['Free Text'].str.replace(a, b)
    

In [110]:
# Outcome measures

# Define the two outcome measures
PRO = ['questionnaire','patient diary'] 
pattern = '|'.join(PRO).lower()

# Extract outcome measures from Free Text
df['Outcome Measures'] =  df['Free Text'].str.findall(pattern,
                          flags=re.IGNORECASE).str.join(',').apply(
                          lambda x: ','.join(list(set(str(x).split(',')))))

# Create a binary column to indicate whether the trial uses any of the outcome measures
df['Outcome Measures'] = df['Outcome Measures'].replace('','No',regex = True)
df.loc[df['Outcome Measures']!='No','Outcome Measures (Y/N)'] = 1
df.loc[df['Outcome Measures (Y/N)']!=1,'Outcome Measures (Y/N)'] = 0

# Create dummy variables for each outcome measure
df = pd.concat([df, df['Outcome Measures'].str.get_dummies(sep=',').add_prefix('Outcome Measures - ')], axis = 1)


In [111]:
# Intervention Procedures

# Define the intervention procedures
interventionprocedures = ['blood collection','surgery','physical examination','magnetic resonance imaging',
                          'electrocardiogram','echocardiogram','urinalysis','xray','biopsy',
                          'pulmonary function test','ultrasound','electroencephalogram','6 minute walk',
                          'pregnancy test','implantation','aphaeresis','endoscopy','cognitive test','lumbar puncture',
                          'rehabilitation']       

pattern = '|'.join(interventionprocedures).lower()

# Extract the intervention procedures from Free Text
df['INTERVENTION Procedure'] = df['Free Text'].str.findall(pattern,flags = re.IGNORECASE
                              ).str.join(',').apply(lambda x: ','.join(list(set(str(x).split(',')))))

# Create binary columns for each intervention procedure
def func_(col,col_new):
    df[col] = df[col].fillna('').replace('','No',regex = True)
    df.loc[df[col].astype(str).str.contains('No'),col_new] = 0
    df.loc[df[col_new] != 0,col_new] = 1

func_('INTERVENTION Procedure','INTERVENTION Procedure (Y/N)')
func_('INTERVENTION Biological','INTERVENTION Biological (Y/N)')
func_('INTERVENTION Drug','INTERVENTION Drug (Y/N)')
func_('INTERVENTION Diagnostic Test','INTERVENTION Diagnostic Test (Y/N)')
func_('INTERVENTION Device','INTERVENTION Device (Y/N)')
func_('INTERVENTION Dietary Supplement','INTERVENTION Dietary Supplement (Y/N)')
func_('INTERVENTION Radiation','INTERVENTION Radiation (Y/N)')
func_('INTERVENTION Behavioral','INTERVENTION Behavioral (Y/N)')

# Create dummy variables for each intervention procedure
df = pd.concat([df, df['INTERVENTION Procedure'].str.get_dummies(sep = ',').add_prefix('INTERVENTION Procedure - ')], axis = 1)


In [112]:
# Drug Delivery Methods

# Define the drug delivery methods
drug_delivery = ['nasal','inhalation','oral','vaginal','transdermal','otic',
                 'injection','buccal','sublingual','ocular','nebulization','cutaneous']

drug_delivery_methods = '|'.join(drug_delivery).lower()

# Extract the drug delivery method from Free Text
df['Drug/Biological Delivery'] =  df['Free Text'].str.findall(drug_delivery_methods,
                                  flags = re.IGNORECASE).str.join(',').apply(
                                  lambda x: ','.join(list(set(str(x).split(',')))))

# Ensure that only trials with drug or biological interventions are flagged with drug delivery methods
df['Drug/Biological Delivery'] = df['Drug/Biological Delivery'].mask((df['INTERVENTION Biological (Y/N)'] == 0) & (
                                 df['INTERVENTION Drug (Y/N)'] == 0), 'Nil')     

# Create dummy variables for each drug delivery method
df = pd.concat([df, df['Drug/Biological Delivery'].str.get_dummies(sep = ',').add_prefix('Drug/Biological Delivery - ')], axis = 1)


In [113]:
# Placebo trials 

# Identify the clinical trials with placebo
df.loc[(df['Free Text'].str.contains('placebo|place bo|vehicle|sugar pill', case = False)),'Placebo (Y/N)'] = 1
df.loc[df['Placebo (Y/N)'] != 1,'Placebo (Y/N)'] = 0


In [114]:
# Make a copy of the notebook
df2 = df.copy()

# De-list the columns
for c in df2.columns:
    df2[c] = df2[c].apply(de_list)

#### 2.3.3  Finalize the dataset for Exploratory Data Analysis

In [115]:
# Replace blank cells with appropriate values
def fillna(columns,value):
    for col in columns:
        df2[col] = df2[col].fillna('').replace('',value,regex = True)

# Fill null with 'Not Provided'
fillna(['Drug/Biological Delivery','Contacts','Primary Purpose','Study Arms',
        'Clinical Trial Sites','Phase','Allocation','Intervention Model',
        'Masking'],'Not Provided')

# Fill null with 'Others'
fillna(['Disease Type'],'Others')


In [116]:
# Make a copy of the dataset
For_EDA = df2.copy()

# Drop irrlelvant columns
# Remove them because the study duration and Year of clinical trial has been identified 
removecol(For_EDA,"Actual Study Start Date")
removecol(For_EDA,"Study Start Date")
removecol(For_EDA,"Estimated Study Start Date")
removecol(For_EDA,"Actual Study Completion Date")
removecol(For_EDA,"Study Completion Date")
removecol(For_EDA,"Estimated Study Completion Date")
removecol(For_EDA,"Estimated Primary Completion Date")
removecol(For_EDA,"Actual Primary Completion Date")
removecol(For_EDA,'date difference')
removecol(For_EDA,'Actual/Estimated Primary Completion Date')

# Remove them because they are irrelevant to our study 
removecol(For_EDA,'Unnamed: 0')
removecol(For_EDA,'Other Study ID Numbers') 
removecol(For_EDA,'Administrative Information')
removecol(For_EDA,'Data element required by the  International Committee of Medical Journal Editors and the  World Health Organization ICTRP')
removecol(For_EDA,'Includes publications given by the data provider as well as publications  identified by ClinicalTrials.gov Identifier')
removecol(For_EDA,'Change History')
removecol(For_EDA,'First Submitted Date')
removecol(For_EDA,'First Posted Date')
removecol(For_EDA,'Last Update Posted Date')
removecol(For_EDA,'Removed Location Countries')
removecol(For_EDA,'Publications')
removecol(For_EDA,'Biospecimen')
removecol(For_EDA,'Verification Date')
removecol(For_EDA,'Results First Posted Date')
removecol(For_EDA,"IPD Sharing Statement")
removecol(For_EDA,"U.S. FDAregulated Product")
removecol(For_EDA,"Has Data Monitoring Committee") 
removecol(For_EDA,'Target FollowUp Duration')   
removecol(For_EDA,'Sampling Method')  
removecol(For_EDA,'Study Population')  
removecol(For_EDA,'Study Groups/Cohorts')
removecol(For_EDA,'link')
removecol(For_EDA,'PRS Account')
removecol(For_EDA,'Accepts Healthy Volunteers')                                
removecol(For_EDA,'Contacts')  

# Remove this because the minimum and maximum age of participants have been identified
removecol(For_EDA,'Ages')
removecol(For_EDA,'Eligibility Criteria')

# Remove this because the number of arms have been identified
removecol(For_EDA,'Arm Type')
                   
# Remove this because the Intervention Types have been identified
removecol(For_EDA,'Intervention Details')

# Remove them because they have been aggregated to a new column namely <Free Text>
removecol(For_EDA,'Current Primary Outcome Measures') 
removecol(For_EDA,'Original Primary Outcome Measures') 
removecol(For_EDA,'Current Secondary Outcome Measures')
removecol(For_EDA,'Original Secondary Outcome Measures') 
removecol(For_EDA,'Current Other Prespecified Outcome Measures') 
removecol(For_EDA,'Original Other Prespecified Outcome Measures') 
removecol(For_EDA,'Official Title') 
removecol(For_EDA,'Brief Summary')
removecol(For_EDA,'Detailed Description')
removecol(For_EDA,"Free Text_") 

# Remove this because the number of sites have been identified 
removecol(For_EDA,'Clinical Trial Sites')


In [117]:
interventional_studies = For_EDA[For_EDA['Study Type'] == 'Interventional']
print('Number of Interventional Clinical Trials: ', len(interventional_studies))

observational = For_EDA[(For_EDA['Study Type'] == 'Observational')|(For_EDA['Study Type'] == 'Observational [Patient Registry]')]
print('Number of Observational Clinical Trials: ', len(observational))

print('Total Number of Clinical Trials: ', len(For_EDA))

Number of Interventional Clinical Trials:  3831
Number of Observational Clinical Trials:  690
Total Number of Clinical Trials:  4521


In [118]:
# check if there are missing values in the dataset
print('Number of missing values:')
interventional_studies.isnull().sum()

Number of missing values:


TRIAL ID (NCT number)                     0
SPONSOR                                   0
STATUS (active/recruiting)                0
CONDITION/DISEASE                         0
Intervention Types                        0
                                         ..
Drug/Biological Delivery - otic           0
Drug/Biological Delivery - sublingual     0
Drug/Biological Delivery - transdermal    0
Drug/Biological Delivery - vaginal        0
Placebo (Y/N)                             0
Length: 230, dtype: int64

In [119]:
# Export the datasets
For_EDA.to_excel('/project/Python/clinical trials_v2.xlsx')

# Export the dataset consisting of interventional studies only
interventional_studies.to_excel('/project/Python/interventional_studies.xlsx')